In [2]:
from collections import Counter, defaultdict
from itertools import chain, combinations
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from scipy.sparse import csr_matrix
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
import string
import tarfile
import urllib.request

def token_features(tokens, feats):
#    for f in tokens:
#        feats['token=' + f] = feats['token=' + f] + 1
#    keys = abc.keys()
#    vals = abc.values()
    
#    for key in keys:
#        newkey.append('token='+key)
        
    
        
#    print(sorted(feats.items()))
#    feats = dict(c)
#    for w in tokens:
#        feats['token='+w] = tokens.count(w)
    tok = tokens.tolist()
    tok = [x.lower() for x in tokens.tolist()]
    for x in tok:
        feats['token='+x] = tok.count(x)
    
feats = defaultdict(lambda: 0)    
token_features(np.array(['hi', 'there', 'hi']), feats)
sorted(feats.items())

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[('token=hi', 2), ('token=there', 1)]

In [4]:
def token_pair_features(tokens, feats, k=3):
#    chunks = [tokens[x:x+k] for x in range(0, len(tokens), k)]
    x = 0
    chunks = []
    combo = []
    keys = []
    vals = []
    final = []
    count = Counter()
    while (k <= len(tokens)):
        chunks.append(tokens[x:k])
        x = x + 1
        k = k + 1
        
    for x in chunks:
        combo.append(sorted(list(combinations(x,2))))
        
    for c in (sorted(combo)):
        count = count + Counter(c)

    d = dict(count.items())
    
    for v in (sorted(d.items(), key=lambda x: x[0], reverse = False)):
        keys.append(v[0])
        vals.append(v[1])
    
    for k in keys:
        final.append(k[0]+"_"+k[1])
    
    for x in range(0,len(vals)):
        feats["token_pair="+final[x]] = vals[x]
    
        

feats = defaultdict(lambda: 0)
token_pair_features((np.array(['a', 'b', 'c', 'd'])), feats)


In [19]:
def tokenize(doc, punct = False):
#    w = doc.split("\'")
#    word = [''.join(x) for x in w]
#    for words in w:
#        word.append(re.findall(r"[\w']+", words))
    word = []
    doc = doc.lower()
    if punct == True:
        word = re.sub('\W+', ' ', doc.lower()).split()
#        word = re.sub("[\W]+", " ", doc).split()
    if punct == False:
#        word = re.findall(r"[\w']+", doc.lower())
        word = [word.strip(string.punctuation) for word in doc.split()]
    return np.array(word)
          
tokenize(" Hi there! Isn't this fun?", punct=True)
    

array(['hi', 'there', 'isn', 't', 'this', 'fun'], 
      dtype='<U5')

In [7]:
word = "isn't"
w = word.split("\'")
x = [" ".join(x) for x in w]
print(len(x))

2


In [8]:
neg_words = set(['bad', 'hate', 'horrible', 'worst', 'boring'])
pos_words = set(['awesome', 'amazing', 'best', 'good', 'great', 'love', 'wonderful'])

def lexicon_features(tokens, feats):
    pos = 0
    neg = 0
    feats["neg_words"] = neg
    feats["pos_words"] = pos
    for words in tokens:
        if words.lower() in neg_words:
            neg = neg + 1
            feats["neg_words"] = neg
        if words.lower() in pos_words:
            pos = pos + 1
            feats["pos_words"] = pos
            
feats = defaultdict(lambda: 0)
lexicon_features(np.array(['i', 'LOVE', 'this', 'great', 'boring', 'movie']), feats)
print(sorted(feats.items()))
        

[('neg_words', 1), ('pos_words', 2)]


In [9]:
def featurize(tokens, feature_fns):
    feats = defaultdict(lambda: 0)
    for f in feature_fns:
        f(tokens,feats)
    return sorted(feats.items())
    
feats = featurize(np.array(['i', 'LOVE', 'this', 'great', 'movie']), [token_features, lexicon_features])
feats

[('neg_words', 0),
 ('pos_words', 2),
 ('token=great', 1),
 ('token=i', 1),
 ('token=love', 1),
 ('token=movie', 1),
 ('token=this', 1)]

In [10]:
def vectorize(tokens_list, feature_fns, min_freq, vocab=None):
    
    rows = list()
    cols = list()
    res = list()
    flist = []
    data = list()
    c = Counter()
    feats = defaultdict(lambda: 0)
    vocabs = defaultdict()
    for tokens in tokens_list:
        flist.append(featurize(tokens,feature_fns))
    for f in sorted(flist):
        c[f[0]] += 1
    
    if vocab == None:
        for i,j in enumerate(data):
            vocabs[j] = i
            
    for m,n in enumerate(tokens_list):
        feats = featurize(n, feature_fns)
        for f in feats:
            if f[0] in vocabs:
                rows.append(index)
                cols.append(vocab[f[0]])
                res.append(f[1])
                
    row = np.array(rows)
    column = np.array(cols)
    dat = np.array(res)
    
    return (csr_matrix((dat, (row, column)), shape=(len(tokens_list), len(vocabs)), dtype=np.int64))

            
docs = ["Isn't this movie great?", "Horrible, horrible movie"]
tokens_list = [tokenize(d) for d in docs]
feature_fns = [token_features]
vectorize(tokens_list, feature_fns, min_freq=1)

<2x0 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [22]:
def vector(tokens_list, feature_fns, min_freq, vocab=None):
    vocabulary = {}
    indptr = [0]
    data = []
    indices = []
    for token in tokens_list:
        for t in token:
            index = vocabulary.setdefault(t, len(vocabulary))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    return (csr_matrix((data, indices, indptr), dtype=int).toarray())

docs = ["Isn't this movie great?", "Horrible, horrible movie"]
tokens_list = [tokenize(d) for d in docs]
feature_fns = [token_features]
vector(tokens_list, feature_fns, min_freq=1)

array([[1, 1, 1, 1, 0],
       [0, 0, 1, 0, 2]])

In [484]:
lis=[]
    tbool=[]
    combs=[]
    tbool.append([tokenize(d,keep_internal_punct=True) for d in docs])
    tbool.append([tokenize(d,keep_internal_punct=False) for d in docs])
    for i in range(1, len(feature_fns)+1):
        els = [list(x) for x in combinations(feature_fns, i)]
        combs.extend(els)
        
    for i in range(0,len(tbool)):
        for m in min_freqs:
            for f in combs:
                S,vocab=vectorize(tbool[i], f, m)
                accuracy = cross_validation_accuracy(LogisticRegression(), S, labels, 5)
                if i==0:
                    dic={"punct": True,"features" : f,"min_freq":m,"accuracy":accuracy}
                else:
                    dic={"punct": False,"features" : f,"min_freq":m,"accuracy":accuracy}
                lis.append(dic)
    return sorted(lis, key=lambda k: -k['accuracy'])

TypeError: no supported conversion for types: (dtype('O'),)

In [705]:
def fit_best_classifier(docs, labels, best_result):    
    for d in docs:
        tokens = tokenize(d,best_result["punct"])
        X,vocab=vectorize(tokens,best_result["features"] , nest_result["min_freq"])
        best_classifier = LogisticRegression()
    return (best_classifier.fit(X,labels),vocab)

In [637]:
def accuracy_score(truth, predicted):
    return len(np.where(truth==predicted)[0]) / len(truth)

In [648]:
def cross_validation_accuracy(clf, X, labels, k):
    kfold = KFold(n_splits=k)
    act_acc = list()
    for train, test in kfold.split(X):
        pred_acc = clf.predict(X[test])
        clf.fit(X[train], labels[train])
        calc_acc = accuracy_score(labels[test], pred_acc)
        act_acc += acc
    return (np.mean(act_acc))


In [669]:
def eval_all_combinations(docs, labels, punct_vals,
                          feature_fns, min_freqs):
    clf = LogisticRegression()
    i = 0
    result = dict()
    eval_list = list()
    features = []
    for i in range(1, len(feature_fns) + 1):
        features.append(combinations(feature_fns, i))
    for feature in features:
        for punct in punct_vals:
            for min_f in min_freqs:
                tokens = [tokenize(doc, keep_internal_punct=punct) for doc in docs]
                X, vocab = vectorize(tokens, feature, min_freq=min_f)
                act_acc = cross_validation_accuracy(clf, X, labels, 5)
                result['punct'] = punct
                result['features'] = feature
                result['min_freq'] = min_f
                result['accuracy'] = act_acc
                eval_list = eval_list + result
                
    results = sorted(eval_list, key=lambda x: x['accuracy'], reverse = False)
    return results

In [671]:
def plot_sorted_accuracies(results):
    acc = []
    for r in results:
        acc.append(r['accuracy'])
        
    plt.plot(acc)
    plt.ylabel('ACC')
    plt.show()
    plt.savefig("accuracies.png")
        

In [693]:
def mean_accuracy_per_setting(results):
    min_freq = list()
    punct = list()
    feats = list()
    values = list()
    acc = list()
    mean_acc = list()
    for result in results:
        min_freq.append(("min_freq", result['min_freq']))
        punct.append(("punct", result['punct']))
        feats.append(("features", result['features']))
    values.append(min_freq)
    values.append(punct)
    values.append(feats)
    for val in values:
        for result in results:
            if result[values[0]]==values[1]:
                acc.append(result['accuracy'])
                
        for freq in values[1]:
            setting = "=" + " ".join(i.__name__)
        if values[0] == "features":
            setting = values[0] + setting 
        else:
            setting = values[0] + "=" + str(values[1])
    
    mean = sum(acc)/len(acc)
    mean_acc.append((mean,setting))
    return sorted(mean_acc, key=lambda x:x[0], reverse = False )

In [708]:
def top_coefs(clf, label, n, vocab):
    coef = clf.coef_[0]
    poslist = list()
    neglist = list()
    vocabs = sorted(vocab.keys())
    vocab_coef = zip(sorted_vocab, coeff)
    if label == 1:
        for z in vocab_coef:
            if z[1] >= 0:
                poslist.append(z)
        result = sorted(poslist, key=lambda x: x[1], reverse = True)
        return result[0:n]
    if label == 0:
        for z in vocab_coef:
            if z[1] < 0:
                neglist.append(z)
        result = sorted(neglist, key=lambda x: x[1], reverse = False)
        return result[0:n]
        

    

In [711]:
def parse_test_data(best_result, vocab):
    docs, labels = read_data(os.path.join('data', 'test'))
    for doc in docs:
        tokens = [tokenize(doc,best_result["punct"])]
        
    cfr,vocabulary = vectorize(tokens, best_result["features"], best_result["min_freq"], vocab)
    return(docs,labels,cfr)

In [712]:
def print_top_misclassified(test_docs, test_labels, X_test, clf, n):
    predict = clf.predict(X_test)
    probability = clf.predict_proba(X_test)
    actual_prob = difaultdict()
    result = list()
    for x,y in enumerate(predict):
        if y == test_labels[x]:
            continue
        else:
            actual_prob[x] = probability[x][y]
    temp = sorted(actual_prob.items(), key=lambda x: x[1], reverse = True)
    for x in range(0,n):
        result.append(temp)
    for var in result:
        print("Actual Truth = " , test_labels[i[0]], " ", "Predicted Truth = ", predict_value[i[0]], " ",
              "Probability = ", str(i[1]))
        print(test_docs[i[0]])
        